# Markov Chain Monte Carlo

Let's consider a quantity $O$ which can be computed as:

$$
O = \sum\limits_{x \in \chi} o \left( x \right) \tilde{f} \left( x \right) \ ,
$$

where $x$ is a (possibly multidimensional) discrete variable on a state space $\chi$ which must be summed over, while $\tilde{f}$ is a probability density function on $\chi$, so that $\sum\limits_{x \in \chi} \tilde{f} \left( x \right) = 1$. From now on, we define $\tilde{f}$ as the $\textbf{target}$ distribution and we omit the state space label $\chi$. Since the target distribution is normalized, we can write:

$$
\tilde{f} \left( x \right) \equiv \frac{f \left( x \right)}{\sum\limits_{x} f \left( x \right)} \ ,
$$

from which:

$$
O = \frac{ \sum\limits_{x} f \left( x \right) o \left( x \right) }{\sum\limits_{x} f \left( x \right)} \ .
$$

If the target distribution is computable up to a multiplying constant, the $\textbf{Metropolis–Hastings algorithm}$ allows to construct on state space $\chi$ an ergodic Markov chain with length $N_{MC}$: $x_{1} , \ x_{2} , \ x_{3} \dots , x_{n} \ \dots , x_{N_{MC}}$ such that $x_{n}$ is converging (in distribution) to $\tilde{f}$, exploring the space $\chi$ progressively. If we define:

$$
O_{N_{MC}} = \frac{1}{N_{MC}} \sum\limits_{i = 1}^{N_{MC}} o \left(x_i \right) \ ,
$$

then, since the chain can be considered as a statistical sample, the law of large numbers ensures that:

$$
\lim_{N_{MC} \to \infty} O_{N_{MC}}  = O \ ,
$$

which allows us to write:

$$
O_{N_{MC}} \approx O \hspace{3mm} \textrm{for $N_{MC} \gg 1$} \ .
$$

Namely, we end up with an estimate of the original sum. Since the simulation is Markovian and the chain itself can be considered as a statistical sample, the latter usually depends on the starting value. The initial steps are tipically removed as $\textbf{burn-in}$, when the chain is still in the so-called $\textbf{thermalization}$ phase.

In order to transit from the value $x_{n}$ of the chain to the value $x_{n + 1}$, we require a $\textbf{proposal distribution}$ $q$ defined on space $\chi$. If $q$ is positive everywhere, then the Metropolis-Hastings algorithm preserves $\tilde{f}$ as the stationary distribution to which the chain is progressively converging. 

In the $\textbf{random walk Metropolis-Hastings}$, the proposal distribution consists in a local exploration of the neighborhood of the current value $x_n$ of the
Markov chain. That is, the proposed value $y_n$ is simulated as:

$$
y_n = x_n + \epsilon_n \ ,
$$

where $\epsilon_n$ is a random perturbation with distribution $g$. As proposal distribution $g$, we focus on a discrete truncated normal distribution centered on $x_n$. A few relevant definitions are discussed next.

Finally, since $x_n$ depends on the previous element along the Markov chain, this induces a non-zero correlation between $x_n$ and $x_{n+d}$. The correlation between $x_n$ and $x_{n+d}$ is defined as the $\textbf{autocorrelation}$ at lag $d$. For a Markov chain that converges to a stationary distribution, the autocorrelation should indeed decrease as the lag increases. The definition of the autocorrelation function between the chain elements $x_n$ and $x_{n+d}$ is defined as the expected value $\mathbb{E} \left(x_n x_{n+d} \right) $.

### Truncated normal distribution

The probability density function of a normal distribution with mean $0$ and standard deviation $\sigma$ is defined as:  

$$
\mathcal{N}(x, \sigma) = \frac{1}{\sigma \sqrt{2 \pi}}e^{-\frac{x^2}{2 \sigma^2}} \ ,
$$

where $x \in \mathbb{R}$. The probability density function of a normal distribution with mean $\mu$ is simply given by $\mathcal{N}(x - \mu, \sigma)$ .


In the discrete (integer) case, we can define the probability density function of a normal distribution with mean $0$ and standard deviation $\sigma$ as:

$$
\mathcal{N}_{d}(n, \sigma) = \Phi ( n + 0.5, \sigma ) - \Phi (n - 0.5, \sigma ) \ ,
$$

where $n \in \mathbb{N}$ and $\Phi ( x, \sigma ) $ is the cumulative distribution function of a normal distribution with mean $0$ and standard deviation $\sigma$, defined as:

$$
\Phi (x , \sigma) = \frac{1}{\sigma \sqrt{2 \pi}} \int_{- \infty}^{x} e^{-\frac{t^2}{2 \sigma^2}} dt \ .
$$

For convenience, let's also define:

$$
\Phi (a, b; \sigma) = \Phi (b, \sigma) - \Phi (a, \sigma) = \frac{1}{\sigma \sqrt{2 \pi}} \int_{a}^{b} e^{-\frac{t^2}{2 \sigma^2}} dt \ .
$$

The cumulative distribution function of a discrete (integer) gaussian is written as:

$$
\Phi_{d} (n_1, n_2; \sigma) = \mathcal{N}_{d}(n_1, \sigma) + \mathcal{N}_{d}(n_1 + 1, \sigma) + \dots + \mathcal{N}_{d}(n_2, \sigma) \ .
$$

With the above definitions, we can define the probability density function of a truncated discrete (integer) normal distribution between $n_1$ and $n_2$ as:

$$
\mathcal{N_{d, t}}(n, n_1, n_2; \sigma) = \frac{\mathcal{N_{d}}(n, \sigma)}{\Phi_{d} (n_1, n_2; \sigma)} \ .
$$

# Single vertex random walk

Here I build a version of random walk over LQG intertwiners of a single 4-simplex. Each process runs an independent Markov Chain

In [ ]:
using JupyterFormatter
enable_autoformat()

In [1]:
using Distributed

In [3]:
if (nprocs() == 1)
    addprocs(4)
end
println("there are $(nprocs()) processes and $(nworkers()) workers")

there are 5 processes and 4 workers


In [6]:
@everywhere using HalfIntegers, JLD2, ElasticArrays, Distributions, Random, StatsBase

In [19]:
@everywhere function random_walk_function(
    j::Float64,
    D::Int64,
    d::Int64,
    A::Array{Float64,5},
    N::Int64,
    b::Int64,
    σ::Float64,
    draws_folder::String,
    chain_id::Int64,
    model::String,
    verbosity::Int64,
)

    draws = ElasticArray{Int64}(undef, 6, 0)     # some allocations    
    ampls = ElasticArray{Float64}(undef, 0)

    Truncated_Coefficients = zeros(Float64, D)    # some allocations   
    Normal_distribution = Normal(0, σ)

    for i = 0:d
        r = 0.0
        for n = (-i):1:(d-i)
            r += (cdf(Normal_distribution, n + 0.5) - cdf(Normal_distribution, n - 0.5))
        end
        Truncated_Coefficients[i+1] = r
    end

    if (myid() == 1)
        if (verbosity > 1)
            println("Truncated coefficients for j=$(j) are $(Truncated_Coefficients)\n")
        end
    end

    #Initial draw and gaussian 
    draw = Array{Int64}(undef, 6)           # some allocations --- # 1 final slot for molteplicity
    gaussian_draw = Array{Int64}(undef, 5)  # some allocations 

    amp = 0.0
    while (amp == 0)
        for i = 1:5
            draw[i] = rand((1:D))     # some allocations        
        end
        amp = A[draw[1], draw[2], draw[3], draw[4], draw[5]]
    end

    draw[6] = 1 # Initial molteplicity      

    if (myid() == 1)
        if (verbosity > 1)
            println("Initial draw is $(draw[1:5]) with amp $(amp)\n")
        end
    end

    #Proposed draw  
    prop_draw = Array{Int64}(undef, 5)      # some allocations

    acceptance_ratio = 0
    molteplicity = 1                              # initial molteplicity counter
    draw_float_sample = Array{Float64}(undef, 1)   # some allocations --- Distribution pkg does not allow it to be scalar without allocating memory     

    RW_monitor = true  # to test if the RW actually moved

    for n = 1:1:N

        RW_monitor = true

        # Sampling proposed draw   
        Cx = Cx_prop = 1.0
        for i = 1:1:5
            while true
                rand!(Normal_distribution, draw_float_sample)
                @inbounds gaussian_draw[i] = round(Int64, draw_float_sample[1])
                @inbounds prop_draw[i] = draw[i] + gaussian_draw[i]
                @inbounds !(1 <= prop_draw[i] && prop_draw[i] <= (D)) || break
            end
            if (gaussian_draw[i] != 0)
                RW_monitor = false
            end
            @inbounds Cx *= Truncated_Coefficients[draw[i]]
            @inbounds Cx_prop *= Truncated_Coefficients[prop_draw[i]]
        end

        if (RW_monitor == true)

            if (myid() == 1)
                if (verbosity > 1)
                    println(
                        "Iteration $(n)---------------------------------------------------------------\n",
                    )
                    println(
                        "The prop_draw below:\n$(prop_draw[1:5])\nturns out to be equal to the current draw:\n$(draw[1:5])\nso that the molteplicity of the current draw is raised to $(molteplicity + 1)\n",
                    )
                end
            end

            acceptance_ratio += 1
            molteplicity += 1
            continue

        else

            if (myid() == 1)
                if (verbosity > 1)
                    println(
                        "Iteration $(n)---------------------------------------------------------------\n",
                    )
                    println(
                        "draw is:\n$(draw[1:5])\nprop_draw is:\n$(prop_draw[1:5])\namp is $(amp)\n",
                    )
                end
            end


            Prop_amp =
                A[prop_draw[1], prop_draw[2], prop_draw[3], prop_draw[4], prop_draw[5]]

            p = min(1.0, (((Prop_amp^2) / (amp^2))) * (Cx / Cx_prop))

            if (isnan(p))
                error(
                    "got NaN while computing densities ratio: prop_draw = $(prop_draw), amp = $(amp)",
                )
            end

            r = rand()

            if (r < p)

                if (myid() == 1)
                    if (verbosity > 1)
                        println(
                            "prop_draw $(prop_draw) was accepted, since p=$(p) and r=$(r)\n",
                        )
                    end
                end

                if (n > b)
                    # TODO avoid resizing at every iteration (pre-allocating assuming >~ 30% accept. rate and re-allocated only at the end) 
                    # and eventually replace matrices with 1-d arrays (more efficient?)
                    resize!(draws, 6, size(draws)[2] + 1)     # some allocations
                    resize!(ampls, size(ampls)[1] + 1)         # some allocations
                    draw[6] = molteplicity
                    draws[:, end] = draw[:]
                    ampls[end] = amp

                    if (myid() == 1)
                        if (verbosity > 1)
                            println(
                                "The old draw $(draw[1:5]) has been stored with molteplicity $(draw[6])\nthe corresponding amplitude $(amp) has been stored as well",
                            )
                        end
                    end

                end

                molteplicity = 1

                for i = 1:5
                    draw[i] = prop_draw[i]
                end

                amp = Prop_amp
                acceptance_ratio += 1

                if (myid() == 1)
                    if (verbosity > 1)
                        println("Now the new draw is $(draw[1:5])\nthe new amp is $(amp)\n")
                    end
                end
            else
                molteplicity += 1
                if (myid() == 1)
                    if (verbosity > 1)
                        println(
                            "prop_draw $(prop_draw) was rejected, since p=$(p) and r=$(r)\nThe current draw $(draw[1:5]) remains the same and its molteplicity is $(molteplicity)\namp remains $(amp)\n",
                        )
                    end
                end
            end # if condition  (r<p)

        end # if condition  RW_monitor == true  

        # final storage  
        if (n == N)
            resize!(draws, 6, size(draws)[2] + 1)     # some allocations 
            resize!(ampls, size(ampls)[1] + 1)         # some allocations 
            draw[6] = molteplicity
            draws[:, end] .= draw[:]
            ampls[end] = amp
            if (myid() == 1)
                if (verbosity > 1)
                    println(
                        "The last draw $(draw[1:5]) has been stored with molteplicity $(draw[6])\n",
                    )
                end
            end
        end

    end # n cycle      

    #This is such that draws has structure: [N,6], so the operators will be computed faster  
    draws = transpose(draws)  # some allocations  

    if (chain_id == 1)
        println(
            "Done! $(acceptance_ratio*100/N)% of proposed draws have been accepted in master chain",
        )
    end

    draws_number = size(draws)[1]

    if (chain_id == 1)
        if (verbosity > 0)
            println("$(draws_number) draws stored in master chain")
        end
    end

    @save "$(draws_folder)/draws_chain=$(chain_id).jld2" draws    # some allocations 
    @save "$(draws_folder)/ampls_chain=$(chain_id).jld2" ampls    # some allocations 

end

In [40]:
@everywhere begin

    N = 10^6
    b = 0
    σ = 0.9
    draws_folder_base = "../../data/MCMC/draws/N_$(N)_b_$(b)_$(σ)"
    chain_id = myid()
    verbosity = 0

    for model in ["BF", "EPRL"]

        for j = 0.5:0.5:6.0

            local draws_folder = "$(draws_folder_base)/$(model)/j_$(j)"
            mkpath(draws_folder)

            D = Int(2j + 1)
            d = Int(2j)

            @load "../data/$(model)_vertices/julia/vertex_j_$(j).jld2" vertex

            @time random_walk_function(
                j,
                D,
                d,
                vertex,
                N,
                b,
                σ,
                draws_folder,
                chain_id,
                model,
                verbosity,
            )

        end

    end

end

Done! 45.1347% of proposed draws have been accepted in master chain
  0.738509 seconds (1.98 M allocations: 141.020 MiB, 2.50% gc time, 72.52% compilation time: 100% of which was recompilation)
Done! 42.1172% of proposed draws have been accepted in master chain
  0.234227 seconds (369.94 k allocations: 63.570 MiB, 6.41% gc time)
Done! 34.4567% of proposed draws have been accepted in master chain
  0.143166 seconds (306.03 k allocations: 56.744 MiB)
Done! 32.9585% of proposed draws have been accepted in master chain
  0.144889 seconds (296.95 k allocations: 55.774 MiB)
Done! 34.7439% of proposed draws have been accepted in master chain
  0.199346 seconds (318.80 k allocations: 58.108 MiB, 6.35% gc time)
Done! 34.2371% of proposed draws have been accepted in master chain
  0.160598 seconds (316.31 k allocations: 57.840 MiB)
Done! 33.4221% of proposed draws have been accepted in master chain
  0.229294 seconds (309.69 k allocations: 57.133 MiB, 4.33% gc time)
Done! 34.7344% of proposed dr

# Computing dihedral angles

Let's compute the expectation value of the dihedral angle on the first node

In [22]:
function from_intertwiner_to_angle(i::Int64, j)
    return (i * (i + 1) - 2j * (j + 1)) / (2j * (j + 1))
end

from_intertwiner_to_angle (generic function with 1 method)

In [27]:
# not optimized

function angles_computation!(
    chains_to_assemble::Int64,
    angle_all_chains,
    angle_numerical_fluctuations,
    model::String,
    j::Float64,
    N,
    b,
    draws_folder::String,
)

    for chain_id = 1:chains_to_assemble

        @load "$(draws_folder)/ampls_chain=$(chain_id).jld2" ampls
        @load "$(draws_folder)/draws_chain=$(chain_id).jld2" draws
        number_of_elements_in_this_chain = size(ampls)[1]

        angle_this_chain = 0.0

        for i = 1:number_of_elements_in_this_chain
            angle_this_chain += from_intertwiner_to_angle(draws[i, 1] - 1, j) * draws[i, 6]
        end

        angle_this_chain /= (N - b)
        angle_all_chains[chain_id] = angle_this_chain

        # average_angle_first_node 
        angle_numerical_fluctuations[1] = mean(angle_all_chains[:])

        # std_dev_angle_first_node 
        angle_numerical_fluctuations[2] = std(angle_all_chains[:])

        # number of combined chains
        angle_numerical_fluctuations[3] = chains_to_assemble

    end

end

angles_computation! (generic function with 1 method)

In [28]:
using CSV, DataFrames

In [41]:
chains_to_assemble = nprocs()

N = 10^6
b = 0
σ = 0.9
draws_folder_base = "../data/MCMC/draws/N_$(N)_b_$(b)_$(σ)"
operator_folder_base = "../data/MCMC/operators/N_$(N)_b_$(b)_$(σ)"

for model in ["BF", "EPRL"]

    chain_id = myid()

    angle_fluctuations_table = DataFrame()

    # creating first column of angles numerical fluctuations dataframe 
    array = String[]
    push!(array, "avg. angles (first node)")
    push!(array, "std. angles (first node)")
    push!(array, "n. chains combined")

    angle_fluctuations_table = DataFrame(numerical_data = array)

    angle_all_chains = zeros(Float64, chains_to_assemble)
    angle_numerical_fluctuations = zeros(Float64, 3)

    for j = 0.5:0.5:6.0

        local draws_folder = "$(draws_folder_base)/$(model)/j_$(j)"

        column_name = "j=$(j)"

        # not optimized
        @time angles_computation!(
            chains_to_assemble,
            angle_all_chains,
            angle_numerical_fluctuations,
            model,
            j,
            N,
            b,
            draws_folder,
        )

        angle_numerical_fluctuations_dataframe =
            DataFrame(to_rename = angle_numerical_fluctuations)
        rename!(angle_numerical_fluctuations_dataframe, :to_rename => column_name)

        angle_fluctuations_table =
            hcat(angle_fluctuations_table, angle_numerical_fluctuations_dataframe)

    end

    local operator_folder = "$(operator_folder_base)/$(model)"
    mkpath(operator_folder)

    CSV.write(
        "$(operator_folder)/angles_numerical_fluctuations_table.csv",
        angle_fluctuations_table,
    )

end

  0.262609 seconds (10.40 M allocations: 278.984 MiB, 6.70% gc time)
  0.278696 seconds (11.07 M allocations: 296.924 MiB, 7.15% gc time)
  0.241452 seconds (9.18 M allocations: 246.362 MiB, 8.66% gc time)
  0.217957 seconds (8.88 M allocations: 238.432 MiB, 4.49% gc time)
  0.246234 seconds (9.50 M allocations: 254.841 MiB, 8.38% gc time)
  0.242434 seconds (9.44 M allocations: 253.386 MiB, 7.68% gc time)
  0.230148 seconds (9.28 M allocations: 249.154 MiB, 4.20% gc time)
  0.247473 seconds (9.72 M allocations: 260.720 MiB, 9.01% gc time)
  0.246514 seconds (9.61 M allocations: 257.920 MiB, 7.82% gc time)
  0.241779 seconds (9.63 M allocations: 258.288 MiB, 4.93% gc time)
  0.262808 seconds (9.86 M allocations: 264.595 MiB, 8.09% gc time)
  0.261526 seconds (9.88 M allocations: 265.087 MiB, 8.55% gc time)
  0.268311 seconds (10.43 M allocations: 279.750 MiB, 4.49% gc time)
  0.286276 seconds (11.00 M allocations: 294.924 MiB, 8.95% gc time)
  0.241538 seconds (9.32 M allocations: 250.